In [ ]:
import (
    "fmt"
    "bufio"
    "strings"
    "os"
    "strconv"
    "math"
)

const MAX_VAL = int(^uint(0)>>1)

type Position struct {
    row int
    col int
}

type Edge struct {
    dest Position
    weight int
}

type Node struct {
    value int
    edges []Edge
    visited bool
    spNeighbor Position
    cost int
}

func loadData(filename string) map[Position]*Node {
    f, err := os.Open(filename)
    input := bufio.NewReader(f)
    scanner := bufio.NewScanner(input)
    i := 0
    mapData := map[Position]*Node{}
    for scanner.Scan() {
        for j,v := range strings.Split(scanner.Text(),"") {
            val,_ := strconv.Atoi(v)
            mapData[Position{i,j}] = &Node{val,[]Edge{}, false, Position{-1,-1}, MAX_VAL}
        }
        i++
    }
    return mapData
}

func addEdge(grid map[Position]*Node, origin Position, dest Position) {
    grid[origin].edges = append(grid[origin].edges,Edge{dest,grid[dest].value})
}

func addAdjacencies(grid map[Position]*Node) {
    for k,v := range grid {
        if grid[Position{k.row -1,k.col}] != nil {
            addEdge(grid, k, Position{k.row-1,k.col})
        }
        if grid[Position{k.row +1,k.col}] != nil {
            addEdge(grid, k, Position{k.row+1,k.col})
        }
        if grid[Position{k.row,k.col-1}] !=  nil {
            addEdge(grid, k, Position{k.row,k.col-1})
        }
        if grid[Position{k.row,k.col+1}] != nil {
            addEdge(grid, k, Position{k.row,k.col+1})
        }                                                          
    }
}

func min(a,b int) int {
    if a < b {
        return a
    }
    return b
}

func calcSP(gridMap map[Position]*Node, origin Position, destination Position) {
    currNode := origin
    gridMap[currNode].visited = true   
    if currNode == destination {
        return
    }
    
    for i,v := range gridMap[currNode].edges {
        if gridMap[currNode].cost + v.weight < gridMap[v.dest].cost {
            gridMap[v.dest].cost = gridMap[currNode].cost + v.weight
            gridMap[v.dest].visited = false //the weight has changed, recalculate neighbor costs
            gridMap[v.dest].spNeighbor = currNode
        }
    }
    for i,v := range gridMap[currNode].edges {
        if !gridMap[v.dest].visited {
            calcSP(gridMap,v.dest,destination)
        }
    }    
}

In [ ]:
// Part 1
gridMap := loadData("input_data")
addAdjacencies(gridMap)

sPos := Position{0,0}
gDim := int(math.Sqrt(float64(len(gridMap))))
dPos := Position{gDim-1,gDim-1}
gridMap[sPos].cost = 0

calcSP(gridMap, sPos, dPos)
fmt.Println("Lowest-cost path costs",gridMap[dPos].cost)

In [ ]:
// Part 2

func expandGrid(gridMap map[Position]*Node,factor int) {
    gDim := int(math.Sqrt(float64(len(gridMap))))
    for i := 0; i < gDim; i++ {
        for j := 0; j < factor-1; j++ {
            cStart := gDim*j
            aStart := (gDim*j)+gDim 
            for k := cStart; k < aStart; k ++ {
                val := gridMap[Position{i,k}].value + 1
                if val == 10 {
                    val = 1
                }
                gridMap[Position{i,k+gDim}] = &Node{val,[]Edge{}, false, Position{-1,-1}, MAX_VAL}
            }
        }
    }

    for i := 0; i < gDim; i++ {
        for j := 0; j < factor-1; j++ {
            cStart := gDim*j
            aStart := (gDim*j)+gDim
            for k := 0; k < gDim*5; k++ {
                val := gridMap[Position{cStart+i,k}].value + 1
                if val == 10 {
                    val = 1
                }
                gridMap[Position{aStart+i,k}] = &Node{val,[]Edge{}, false, Position{-1,-1}, MAX_VAL}
            }
        }
    }
}


gridMap := loadData("input_data")
expandGrid(gridMap,5)
addAdjacencies(gridMap)

sPos := Position{0,0}
gDim := int(math.Sqrt(float64(len(gridMap))))
dPos := Position{gDim-1,gDim-1}
gridMap[sPos].cost = 0

calcSP(gridMap, sPos, dPos)
fmt.Println("Lowest-cost path costs",gridMap[dPos].cost)